# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## GPU Check

First, you'll need to enable GPUs for the notebook:

- Navigate to `Edit`→`Notebook Settings`
- Select T4 GPU from the Hardware Accelerator section
- Click `Save` and accept.

Next, we'll confirm that we can connect to the GPU:

In [6]:
import torch

if not torch.cuda.is_available():
    raise RuntimeError("GPU not available")
else:
    print("GPU is available!")

GPU is available!


## Installing dependencies

In [7]:
%pip install ragatouille PyPDF2

In [8]:
%pip install git+https://github.com/mozilla-ai/structured-qa.git@5-add-benchmark

  Cloning https://github.com/mozilla-ai/structured-qa.git (to revision 5-add-benchmark) to /tmp/pip-req-build-v6q9_weu
  Running command git clone --filter=blob:none --quiet https://github.com/mozilla-ai/structured-qa.git /tmp/pip-req-build-v6q9_weu
  Running command git checkout -b 5-add-benchmark --track origin/5-add-benchmark
  Switched to a new branch '5-add-benchmark'
  Branch '5-add-benchmark' set up to track remote branch '5-add-benchmark' from 'origin'.
  Resolved https://github.com/mozilla-ai/structured-qa.git to commit 97049d67d83ec6129569d442bd365c7a5e490578
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
!wget https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv

--2025-02-04 15:27:16--  https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23304 (23K) [text/plain]
Saving to: ‘structured_qa.csv.1’

structured_qa.csv.1 100%[===================>]  22.76K  --.-KB/s    in 0.001s  

2025-02-04 15:27:17 (28.5 MB/s) - ‘structured_qa.csv.1’ saved [23304/23304]



# Setup

In [10]:
import os
import google.generativeai as genai
from google.colab.userdata import get, SecretNotFoundError

try:
    genai.configure(api_key=get("GOOGLE_API_KEY"))
except SecretNotFoundError as e:
    raise RuntimeError("Please set the GOOGLE_API_KEY secret to your API key") from e
os.environ["LOGURU_LEVEL"] = "INFO"

In [11]:
from loguru import logger

In [12]:
import PyPDF2


def load_pdf(pdf_file: str) -> str | None:
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        return "\n".join(page.extract_text() for page in pdf_reader.pages)
    except Exception as e:
        logger.exception(e)
        return None

## Function to Process all questions for a single Document

In [13]:
import json
import time

from ragatouille import RAGPretrainedModel
from ragatouille.data import CorpusProcessor


def process_document(
    document_file,
    document_data,
    model,
):
    logger.info("Setting up RAG")
    RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
    corpus_processor = CorpusProcessor()
    documents = corpus_processor.process_corpus([load_pdf(document_file)])
    RAG.encode([x["content"] for x in documents])

    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in document_data.iterrows():
        if model.n > 0 and model.n % 9 == 0:
            logger.info("Waiting for 60 seconds")
            time.sleep(60)
        question = row["question"]
        question_part, *options = question.split("?")

        logger.info(f"Question: {question}")
        results = RAG.search_encoded_docs(query=question_part, k=3)
        current_info = "\n".join(result["content"] for result in results)
        logger.info(current_info[:100])

        answer = model.model.generate_content(
            [f"This is the document: {current_info}", question]
        )
        logger.info(answer.text)
        answers[index] = answer.text.strip()
        sections[index] = None
        model.n += 1

    return answers, sections

## Load Model

In [14]:
from structured_qa.model_loaders import load_gemini_model

In [15]:
SYSTEM_PROMPT = """
You are a rigorous assistant answering questions.
You must only answer based on the current information available which is:

```
{CURRENT_INFO}
```

If the current information available not enough to answer the question,
you must return "I need more info" srting and nothing else:

If the current information is enough to answer, you must return one of the following formats:
- YES/NO (for boolean questions)
- Number (for numeric questions)
- Single letter (for multiple-choice questions)
"""

In [16]:
model = load_gemini_model("gemini-2.0-flash-exp", system_prompt=SYSTEM_PROMPT)
model.n = 0

# Run Benchmark

In [17]:
from pathlib import Path
from urllib.request import urlretrieve

import pandas as pd

logger.info("Loading input data")
data = pd.read_csv("structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)
for document_link, document_data in data.groupby("document"):
    logger.info(f"Downloading document {document_link}")
    downloaded_document = Path(f"{Path(document_link).name}.pdf")
    if not Path(downloaded_document).exists():
        urlretrieve(document_link, downloaded_document)
        logger.info(f"Downloaded {document_link} to {downloaded_document}")
    else:
        logger.info(f"File {downloaded_document} already exists")

    answers, sections = process_document(downloaded_document, document_data, model)

    for index in document_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-02-04 15:27:45.015 | INFO     | __main__:<cell line: 0>:6 - Loading input data
2025-02-04 15:27:45.044 | INFO     | __main__:<cell line: 0>:11 - Downloading document https://aiindex.stanford.edu/wp-content/uploads/2024/05/HAI_AI-Index-Report-2024.pdf
2025-02-04 15:27:45.376 | INFO     | __main__:<cell line: 0>:15 - Downloaded https://aiindex.stanford.edu/wp-content/uploads/2024/05/HAI_AI-Index-Report-2024.pdf to HAI_AI-Index-Report-2024.pdf.pdf
2025-02-04 15:27:45.381 | INFO     | __main__:process_document:13 - Setting up RAG
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

...

2025-02-04 15:39:29.014 | INFO     | __main__:process_document:37 - 3
2025-02-04 15:39:29.015 | INFO     | __main__:process_document:29 - Question: How many victory conditions are there?
2025-02-04 15:39:29.033 | INFO     | __main__:process_document:32 - Conquering Middle-earth
If you are present in all 7 regions (with a Fortress and/or at least 1 Unit)
2025-02-04 15:39:30.149 | INFO     | __main__:process_document:37 - 3
2025-02-04 15:39:30.151 | INFO     | __main__:process_document:29 - Question: How many different races are there?
2025-02-04 15:39:30.168 | INFO     | __main__:process_document:32 - • 
 Re
veal tokens to both players. There is no hidden information.Bonus spaces
2 matching Race symb
2025-02-04 15:39:31.232 | INFO     | __main__:process_document:37 - I need more info
2025-02-04 15:39:31.234 | INFO     | __main__:process_document:29 - Question: Which player begins the game? -A: Sauron -B: The Fellowship -C: Other
2025-02-04 15:39:31.251 | INFO     | __main__:process_docu

Encoding 48 documents...


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
100%|██████████| 2/2 [00:00<00:00,  9.87it/s]
2025-02-04 15:40:50.170 | INFO     | __main__:process_document:19 - Predicting
2025-02-04 15:40:50.171 | INFO     | __main__:process_document:24 - Waiting for 60 seconds


Shapes:
encodings: torch.Size([48, 508, 128])
doc_masks: torch.Size([48, 508])
Documents encoded!


2025-02-04 15:41:50.175 | INFO     | __main__:process_document:29 - Question: What is the maximum number of cards a player may acquire during the lookout phase?
2025-02-04 15:41:50.192 | INFO     | __main__:process_document:32 - 13. At the beginning of the game, each player draws 5 cards from their 
Clan deck, looks through the
2025-02-04 15:41:51.507 | INFO     | __main__:process_document:37 - 4
2025-02-04 15:41:51.509 | INFO     | __main__:process_document:29 - Question: Is there a limit to the number of cards a player may have in their hand?
2025-02-04 15:41:51.526 | INFO     | __main__:process_document:32 - NOTE 1:  There’s no limit to the number of cards a player may have 
in their hand.  
NOTE 2:  If the
2025-02-04 15:41:52.763 | INFO     | __main__:process_document:37 - NO

2025-02-04 15:41:52.765 | INFO     | __main__:process_document:29 - Question: Can you raid the locations of a player that has passed during the action phase?
2025-02-04 15:41:52.783 | INFO     | __main__:proc

In [18]:
results = pd.read_csv("results.csv")
for index, result in results.iterrows():
    results.loc[index, "pred_answer"] = result["pred_answer"].strip()
    if result["pred_answer"].startswith(
        (f"-{result['answer']}", f"{result['answer']}")
    ):
        results.loc[index, "pred_answer"] = result["answer"]
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,type,section,question,answer,pred_answer,pred_section
10,10,https://arxiv.org/pdf/1706.03762,Scientific Paper,5.4 Regularization,What was the dropout rate used for the base mo...,0.1,0. 1,NaN
26,26,https://authorsalliance.org/wp-content/uploads...,Techincal Documentation,CHAPTER 5: WHERE DO YOU WANT TO MAKE YOUR WORK...,Are Gold Open Access and Green Open Access mut...,NO,YES,NaN
28,28,https://arxiv.org/pdf/2201.11903,Scientific Report,3.1 Experimental Setup,How many large language models were evaluated?,5,FIVE,NaN
29,29,https://arxiv.org/pdf/2201.11903,Scientific Report,3.1 Experimental Setup,How many benchmarks were used to evaluate arit...,5,FIVE,NaN
33,33,https://arxiv.org/pdf/2201.11903,Scientific Report,3.4 Robustness of Chain of Thought,How many annotators provided independent chain...,3,THREE,NaN
34,34,https://arxiv.org/pdf/2201.11903,Scientific Report,3.2 Results,How many random samples were examined to under...,100,50,NaN
37,37,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE EFFECTS,How many different races are there?,6,I NEED MORE INFO,NaN
42,42,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE COSTS,Can a player pay coins to compensate for missi...,YES,NO,NaN
45,45,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE EFFECTS,Which type of cards provide coins? -A: Gray -B...,B,I NEED MORE INFO,NaN
57,57,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CLEANUP PHASE,Is there a cleanup phase in the final round?,NO,YES,NaN


In [19]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.8640776699029126